In [21]:
import sys
import csv
import time
import re
from urllib.request import urlopen
import json
from pandas.io.json import json_normalize
import pandas as pd, numpy as np
import pickle 
import pprint
import requests
from datetime import date, datetime
from random import *
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys 
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
import os
import datetime
import shutil
import glob
from csv import DictWriter

pp = pprint.PrettyPrinter(indent=4)

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('window-size=1000x500')
options.add_argument("start-maximized")
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument("disable-gpu")

### class & function list
* var list: dataframe, search_keyword
* def get_siteinfo():
* def get_sitexy():
* def modify_table():
* def get_postnum():

In [22]:
class campinginfo(object):
    def __init__(self):
#         self.df = df
#         self.kwe_col = kwd_col
        self.chrome_path = '/usr/local/bin/chromedriver'
        self.options = webdriver.ChromeOptions()
        self.options.add_argument('--headless')
        self.options.add_argument('window-size=1000x500')
        self.options.add_argument("start-maximized")
        self.options.add_argument("disable-infobars")
        self.options.add_argument("--disable-extensions")
        self.options.add_argument("disable-gpu")
        
        self.path1 = "../file/campsite_list_raw.csv"
        self.path2 = "../file/campsite_list_modify.csv"
        self.url_1 = 'https://www.gocamping.or.kr/bsite/camp/info/list.do?pageUnit=2500&searchKrwd=&listOrdrTrget=last_updusr_pnttm&pageIndex=1'
        

    def get_siteinfo(self):
        driver = webdriver.Chrome(self.chrome_path, options=self.options)
        driver.get(self.url_1) 

        time.sleep(10)
        source = driver.page_source 
        data = bs(source, 'html.parser')
        cont = data.findAll("div",{"class":"camp_cont"})

        cols = ['name','url','addr']
        df = pd.DataFrame(columns=cols)
        prefix = 'https://www.gocamping.or.kr'
        for i in range(len(cont)):
            name = cont[i].h2.a.get_text().split("]")[1]
            url_t = cont[i].h2.a['href'].split("&")[0]
            url = ''.join([prefix,url_t])
            addr = cont[i].ul(class_="addr")[0].get_text()

            info = [name, url, addr]
            dict1 = dict(zip(cols, info))
            df = df.append(dict1, ignore_index=True)

        df['name'] = df.name.str.strip()
        driver.quit()
        return df

        
    def get_sitexy(self):
        limit_ = 1000
        driver = webdriver.Chrome(self.chrome_path, options=self.options)
        cols = ['name','addr', 'call_num','coord_x','coord_y']
        df = pd.DataFrame(columns=cols)

        for i in range(1,4):
            url_2 = 'https://www.gocamping.or.kr/bsite/camp/info/list.do?listTy=MAP&pageUnit={limit}&searchKrwd=&listOrdrTrget=last_updusr_pnttm&pageIndex={page}'.format(limit=limit_, page=i)
            driver.get(url_2) 
            time.sleep(3)
            source = driver.page_source 
            data = bs(source, 'html.parser')
            cont = data.findAll("div", {"class":"map_list"})[0]
            ul = cont.ul.select("li[onclick]")

            for j in range(len(ul)):
                camp = ul[j]
                xy = camp['onclick']
                x = re.findall('[0-9]*[0-9]\.[0/-9]*[0-9]', xy)[1]
                y = re.findall('[0-9]*[0-9]\.[0/-9]*[0-9]', xy)[0]
                name = camp.h2.text.split(']')[1].strip()
                addr = camp.find("li", {"class":"addr"}).get_text()
                call_num = camp.find("li", {"class":"call_num"}).get_text() if camp.find("li", {"class":"call_num"}) else ""
                val_list = [name, addr, call_num, x, y]
                result = dict(zip(cols, val_list))
                df = df.append(result, ignore_index=True)
            print(df.tail(2))
        df.to_csv(self.path1)
        driver.quit()
        return df
    
    def modify_table(self, df):
        def name_for_tag(x):
            if x=="Najost Camp": return "나조스트캠핑장"
            elif x=="iFA캠프운악": return "캠프운악"
            elif x=="Black tree": return "블랙트리캠핑장"
            elif x=="영천 구룡산 오토&펜션": return "영천구룡산오토캠핑장"
            elif x=="(주)자연에너지-드림랜드오토캠핑장": return "드림랜드오토캠핑장"
            elif x=="소회산리멍우리협곡관광농원(캠핑장)": return "멍우리협곡캠핑장"
            elif x=="춘천Believing(빌리빙)캠핑장": return "춘천빌리빙캠핑장"
            elif x=="변산반도국립공원 고사포 야영장 (임시)": return "고사포야영장"
            elif x=="A&J오토캠핑장": return "AJ오토캠핑장"
            elif x=="Camp 1950 by 민들레울": return "camp1950"
            elif x=="(주)자연인": return "자연인글램핑"
            elif x=="(주)수동자연마을 힐링별밤수목원캠핑장": return "힐링별밤수목원캠핑장"
            elif x=="stay714": return "스테이714"
            elif x=="선녀와나무꾼": return "선녀와나무꾼캠핑장"
            elif x=="블루스카이": return "블루스카이캠핑장"
            elif x=="CLUB 596": return "클럽596"
            elif x=="비토애": return "비토애글램핑"
            elif x=="(주)연합진흥 경도 글램핑파크": return "경도글램핑파크"
            elif x=="(주)태평소금 천일염 힐링캠프": return "천일염힐링캠프"
            elif x=="누룽지": return "누룽지캠핑장"
            elif x=="포시즌": return "포시즌캠핑장"
            elif x=="마이웨이 리조트 관광": return "마이웨이리조트"
            elif x=="바오바오": return "바오바오글램핑" 
            elif x=="블루마운틴": return "팔공산블루마운틴" 
            elif x=="별헤는 밤": return "별헤는밤캠핑장"
            elif x=="선녀와 나무꾼": return "선녀와나무꾼캠핑장"
            elif x=="테르메덴": return "테르메덴카라반"
            elif x=="캠프통 아일랜드": return "캠프통아일랜드글램핑"
            elif x=="포세이돈": return "포세이돈카라반"# 2266	(주)포세이돈	20904
            elif x=="노을캠핑장": return "강화노을캠핑장"# 1005	노을캠핑장	18276

            elif "&" in x: return re.sub("&","앤", x)
            elif re.search("\(.*\)", x): return re.sub("\(.*\)","", x)
            else: return x
        df["hashtag"] = df["name"].apply(name_for_tag)
        df["hashtag"] = df["hashtag"].apply(name_for_tag)
        df["hashtag"] = df["hashtag"].apply(lambda x: x.replace(" ","").strip())
        to_drop = ['산책','선인장','부에노스아이레스','가온','캐빈','숲속의 작은집','생각속의 집', '풍경']
        df = df[~df.name.isin(to_drop)]
        df['coord_x'] = df['coord_x'].apply(lambda x : float(x))
        df['coord_y'] = df['coord_y'].apply(lambda x : float(x))
        # x, y 좌표 바뀐 캠핑장 수정하기
        # ...
        df.reset_index(drop=True, inplace=True)
        df.to_csv(self.path2)
        return df
    
    def get_postnum(self, df, k=0):
        def append_dict_as_row(file_name, dict_of_elem, field_names):
            # Open file in append mode
            with open(file_name, 'a+', newline='') as write_obj:
                # Create a writer object from csv module
                dict_writer = DictWriter(write_obj, fieldnames=field_names)
                # Add dictionary as wor in the csv
                dict_writer.writerow(dict_of_elem)
    
        cols = list(df.columns) + ['num_index', 'search_order', 'search_rslt', 'url','text','dt']
        df_out = pd.DataFrame(columns=cols)
        today_ymd = datetime.datetime.today() .strftime('%Y%m%d')
        filepath = '../file/camping_' + today_ymd + '_' + f"{k:0>4}" +'.csv'
        df_out.to_csv(filepath, index=False)

        driver = webdriver.Chrome(self.chrome_path, options=self.options)
        url = 'https://www.instagram.com/explore/tags/'+'캠프운악'
        driver.get(url) 

        time.sleep(3)

        t = driver.find_elements_by_css_selector("input[placeholder='Search']")[0]
        print(len(df))

        for i in range(k,len(df)):
            print(i, end=" ")
            t.send_keys(df.iloc[i]['hashtag'])
            time.sleep(2)
            t2 = driver.find_elements_by_class_name("yCE8d")
            if len(t2)>0:
                for j in range(len(t2)):
                    ref = t2[j].get_attribute('href')
                    title = t2[j].text.split("\n")[0]
                    txt = t2[j].find_elements_by_class_name("Fy4o8")[0].text
                    info = list(df.iloc[i,:]) + [i, j, title, ref, txt, today_ymd]
                    dict1 = dict(zip(cols, info))
#                     append_csv(filepath, dict1)
                    append_dict_as_row(filepath, dict1, cols)
                    df_out = df_out.append(dict1, ignore_index=True)
            else:
                info = list(df.iloc[i,:]) + [i, 0, '', '', '', today_ymd]
                dict1 = dict(zip(cols, info))
                append_dict_as_row(filepath, dict1, cols)
                df_out = df_out.append(dict1, ignore_index=True)
            t.clear()
            if i>0 and i%100 == 0:
                print()
                print(df_out.tail(2))
                time.sleep(3)
        driver.quit()
        df_out.head(20)

#         df_out.to_csv('../file/camping_' + today_ymd + '.csv', index=False)
        return df_out

In [23]:
a = campinginfo()

In [24]:
campxy = a.get_sitexy()

            name                           addr      call_num      coord_x  \
998  돌고래 하우스 카라반  제주특별자치도 서귀포시 강정동 이어도로 826-58   064-739-7704  126.4994002   
999      이순신 글램핑             전북 부안군 변산면 궁항로 178  063-582-8845  126.4699688   

        coord_y  
998  33.2327306  
999  35.6065049  
          name                addr      call_num      coord_x     coord_y
1998  캠프통 아일랜드  경기 가평군 청평면 호반로 976  032-585-6000  127.4842010  37.7107945
1999    캠프락캠핑장    가평군 상면 원흥리 514-2                127.3206037  37.8223436
        name                  addr call_num      coord_x     coord_y
2448  별천지야영장    경기 가평군 북면 화악산로 816           127.5471852  37.9328135
2449  다놀자캠핑장  경북 상주시 화서면 문장로 425-5           127.9639491  36.4713360


In [25]:
campxy = a.modify_table(campxy)

In [26]:
campxy.head(10)

,name,addr,call_num,coord_x,coord_y,hashtag
0,캠프도고,충청남도 아산시 선장면 삼봉로 70,041-541-5511,126.8733281,36.7749553,캠프도고
1,병풍바위야영장,강원도 삼척시 산양서원1길 89-0 산양1리마을회관,033-572-8658,129.280254,37.1410517,병풍바위야영장
2,덕풍계곡솔밭야영장,강원도 삼척시 가곡면 풍곡리,010-3714-8577,129.1800740,37.1008122,덕풍계곡솔밭야영장
3,앙티브,경기 가평군 상면 수목원로 262-34,031-584-2400,127.3589249,37.7551124,앙티브
4,삼계리주말농원캠핑장,경북 청도군 운문면 신원3길 74,010-9587-3191,129.0067097,35.676399,삼계리주말농원캠핑장
5,야놀자 캠핑장,경남 하동군 옥종면 옥단로 1795-17,,127.8852610,35.2447534,야놀자캠핑장
6,할매야영장,경남 하동군 청암면 청학로 1825-8,,127.7640257,35.2111212,할매야영장
7,옥수골계곡펜션야영장,전북 무주군 적상면 옥소동길 52,063-324-8833,127.7162935,35.966889,옥수골계곡펜션야영장
8,평창 태기산 캠핑장 펜션,강원도 평창군 봉평면 진조리 338-11,010-3949-9091,128.3027752,37.5677373,평창태기산캠핑장펜션
9,해금강호텔 야영장,경상남도 거제시 남부면 갈곶리 78-3,055-632-1100,128.6745320,34.7366185,해금강호텔야영장


In [42]:
# campxy['coord_x']
campxy['coord_x'] = campxy['coord_x'].apply(lambda x : float(x))
campxy['coord_y'] = campxy['coord_y'].apply(lambda x : float(x))

In [302]:
campxy = pd.read_csv("../file/campsite_list_modify.csv", index_col=[0])

In [ ]:
camp_post = a.get_postnum(campxy, 0)

2414
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 
               name                     addr       call_num     coord_x  \
419  십이선녀탕 통나무집 야영장     강원도 인제군 십이선녀탕길 81-0   010-7687-5945  128.302771   
420       도고온천모두캠핑장  충남 아산시 도고면 도고온천로 207-20  010-2857-2040  126.880020   

       coord_y       hashtag num_index search_order search_rslt  \
419  38.183068  십이선녀탕통나무집야영장        99            0  #십이선녀탕통나무집   
420  36.766040     도고온천모두캠핑장       100            0  #도고온천모두캠핑장   

                                                   url      text        dt  
419  https://www.instagram.com/explore/tags/%EC%8B%...  17 posts  20200705  
420  https://www.instagram.com/explore/tags/%EB%8F%...  28 posts  20200705  
101 102 103 104 105 106 107 108 109 110 1

In [361]:
camp_post = pd.read_csv("../file/camping_20200705_0000.csv")

In [366]:
import re
camp_post['url_type'] = camp_post['url'].str.extract(r'(https:\/\/www.instagram.com\/)(\w*\/)(\w*)(.)')[2]
camp_post['rslt_str'] = camp_post['search_rslt'].str.extract(r'(\#)(.*)')[1]

camp_post['posts'] = camp_post['text'].str.extract(r'(\d|\d*|\d*\,\d*)( posts| post)')[0]
camp_post['posts'] = camp_post['posts'].replace(',', '', regex=True)
camp_post['posts'].fillna(0, inplace=True)
camp_post['posts'] = camp_post['posts'].apply(lambda x : int(x))

camp_post['p_grp'] = camp_post['posts'].\
    apply(lambda x :  0\
          if (x==0) 
          else (100 if (x>=100 and x<500) \
                else (500 if (x>=500 and x<1000) \
                else (1000 if (x>=1000 and x<5000) \
                else (5000 if (x>=5000 and x<10000)\
                     else 10000)))))

camp_tag = camp_post.loc[(camp_post['hashtag']==camp_post['rslt_str']) & (camp_post['url_type']=='tags')]

camp_loc = camp_post.loc[(camp_post['url_type']=='locations')].groupby('num_index').first()
camp_loc.rename(columns = {"url": "url_loc"}, inplace=True)

In [368]:
camp_post[(camp_post['posts']<1000) & (camp_post['posts']>500)]

,name,addr,call_num,coord_x,coord_y,hashtag,num_index,search_order,search_rslt,url,text,dt,url_type,rslt_str,posts,p_round,p_grp
347,호식이 캠핑장,강원도 영월군 주천면 판운리,010-3181-6696,128.318727,37.311611,호식이캠핑장,69,0,#호식이캠핑장,https://www.instagram.com/explore/tags/%ED%98%...,612 posts,20200705,tags,호식이캠핑장,612,600,500
375,자연에 머물다,강원 춘천시 사북면 화악지암길 186,010-8025-6508,127.585466,37.972085,자연에머물다,83,0,#자연에머물다,https://www.instagram.com/explore/tags/%EC%9E%...,601 posts,20200705,tags,자연에머물다,601,600,500
411,스카이글램핑,경기도 남양주시 수동면 지둔리,010-4037-1999,127.296799,37.726593,스카이글램핑,96,0,#스카이글램핑,https://www.instagram.com/explore/tags/%EC%8A%...,912 posts,20200705,tags,스카이글램핑,912,900,500
425,자연속캠핑장,경상남도 밀양시 산내면 용전리 403-1,0507-1409-4571,128.862402,35.551979,자연속캠핑장,103,0,#자연속캠핑장,https://www.instagram.com/explore/tags/%EC%9E%...,665 posts,20200705,tags,자연속캠핑장,665,600,500
461,글램퍼스양평,경기 양평군 단월면 대부록길 38-3,031-772-7179,127.682126,37.579118,글램퍼스양평,112,1,#글램퍼스양평점,https://www.instagram.com/explore/tags/%EA%B8%...,652 posts,20200705,tags,글램퍼스양평점,652,600,500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10995,시공간,전남 여수시 남면 솔고지길 105,NaN,127.726503,34.547958,시공간,2369,6,#시공간초월,https://www.instagram.com/explore/tags/%EC%8B%...,725 posts,20200705,tags,시공간초월,725,700,500
11036,자바카라반,인천 옹진군 영흥면 선재로 61-1,010-4045-7823,126.530395,37.234790,자바카라반,2373,0,#자바카라반,https://www.instagram.com/explore/tags/%EC%9E%...,703 posts,20200705,tags,자바카라반,703,700,500
11053,레이크문캠핑장,경기 포천시 영중면 호국로3296번길 31-46,010-3797-7734,127.265669,38.034067,레이크문캠핑장,2381,0,#레이크문캠핑장,https://www.instagram.com/explore/tags/%EB%A0%...,521 posts,20200705,tags,레이크문캠핑장,521,500,500
11061,몬테리오,강원 홍천군 서면 마곡길 220,033-436-1001,127.579771,37.728713,몬테리오,2385,0,#몬테리오,https://www.instagram.com/explore/tags/%EB%AA%...,966 posts,20200705,tags,몬테리오,966,900,500


In [327]:
print(len(camp_tag))
print(len(camp_loc))

1861
561


In [330]:
camp_merge = pd.merge(campxy, camp_tag[['name', 'addr', 'call_num','posts','dt']], how='left', on = ['name', 'addr', 'call_num'])
camp_merge = pd.merge(camp_merge, camp_loc[['name', 'addr', 'call_num','url_loc']], how='left', on = ['name', 'addr', 'call_num'])
camp_merge['posts'].fillna(0, inplace=True)

camp_merge['dt'] = datetime.datetime.today().strftime('%Y%m%d')
camp_merge['loc_yn'] = camp_merge['url_loc'].apply(lambda x: 0 if pd.isnull(x) else 1)


In [331]:
camp_merge.head()

,name,addr,call_num,coord_x,coord_y,hashtag,posts,dt,url_loc,loc_yn
0,자연과함께 펜션캠핑장,강원 영월군 무릉도원면 무릉법흥로 968-31,010-5015-1366,128.280297,37.344759,자연과함께펜션캠핑장,16,20200707,NaN,0
1,파라다이스캠프,강원 영월군 무릉도원면 무릉법흥로 1128-10,010-5554-1008,128.278680,37.357150,파라다이스캠프,51,20200707,NaN,0
2,연포분교야영장,강원 정선군 신동읍 연포길 544-6,NaN,128.571886,37.251265,연포분교야영장,0,20200707,NaN,0
3,두리캠핑장,인천 계양구 장제로 1145 (귤현동),010-3824-0426,126.763061,37.570524,두리캠핑장,2229,20200707,https://www.instagram.com/explore/locations/78...,1
4,강원도양양캠핑장 포레스트,강원 양양군 현북면 검소길 117,010-2000-4949,128.691604,38.007728,강원도양양캠핑장포레스트,2,20200707,NaN,0


In [332]:
camp_merge.to_csv('../file/camp_merge.csv', index=False)